In [6]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import cv2
import json
import numpy as np

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Torch Dataset

In [13]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)

        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h, w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        kps[::2] = 224.0 / w  #Adjusting X coords
        kps[1::2] = 224.0 / h  #Adjusting Y coords

        return img, kps




In [14]:
train_dataset = KeypointsDataset("data/images", "data/data_train.json")
val_dataset = KeypointsDataset("data/images", "data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

# Creating Model

In [15]:
model = models.resnet50(pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, 14*2) #Replacing last layer of resnet50 with custom layer

d:\Coding\Anaconda3\envs\llms\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Coding\Anaconda3\envs\llms\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [16]:
model = model.to(device)

# Training Model

In [17]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [18]:
epochs = 20
for epochs in range(epochs):
    for i, (img, kps) in enumerate(train_loader):
        img = img.to(device)
        kps = kps.to(device)

        optimizer.zero_grad()
        out = model(img)
        loss = criterion(out, kps)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch: {epochs} Batch: {i} Loss: {loss.item()}")

Epoch: 0 Batch: 0 Loss: 0.12672024965286255
Epoch: 0 Batch: 10 Loss: 0.023096643388271332
Epoch: 0 Batch: 20 Loss: 0.008815448731184006
Epoch: 0 Batch: 30 Loss: 0.003356081433594227


KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), "keypoints_model.pth")